In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1b5CC6G-W_RszA4l3kZ1pQHVSrFdbh7aI", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Pipeline Parallelism: The GPU Assembly Line

*Part 4 of the Vizuara series on 5D Parallelism from Scratch*
*Estimated time: 45 minutes*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/5d-parallelism-from-scratch/practice/4/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
# Setup and imports
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

%matplotlib inline

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

print(f"PyTorch version: {torch.__version__}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
print("\n🚀 Welcome to Pipeline Parallelism from Scratch!")

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

In the previous notebooks, we saw that **Data Parallelism** splits the training data across GPUs, and **Tensor Parallelism** splits individual weight matrices within a layer. But what happens when your model has so many layers that the total stack does not fit on a single GPU — even if each individual layer is small enough?

This is where **Pipeline Parallelism** comes in. The idea is beautifully simple: split the model by **depth**. GPU 0 gets layers 1–8, GPU 1 gets layers 9–16, GPU 2 gets layers 17–24, and GPU 3 gets layers 25–32. Each GPU is a specialist — it only handles its assigned stage.

Every major LLM training run (Llama 3, GPT-4, DeepSeek-V3) uses pipeline parallelism. By the end of this notebook, you will understand exactly how it works, why "bubbles" are the central challenge, and how clever scheduling (GPipe and 1F1B) keeps GPUs busy.

**Here is a preview of what we will build** — an interactive pipeline schedule simulator:

In [ ]:
# 🎯 Teaser: Preview of the final output
# (We will build all of this from scratch below!)

fig, axes = plt.subplots(1, 2, figsize=(16, 4))
P, M = 4, 8

for ax_idx, title in enumerate(["GPipe", "1F1B"]):
    ax = axes[ax_idx]
    grid = np.zeros((P, 2 * M + 2 * (P - 1)))
    ax.imshow(grid, cmap='Greys', aspect='auto', vmin=0, vmax=1)
    ax.set_title(f"{title} Schedule (P={P}, M={M})", fontsize=13, fontweight='bold')
    ax.set_ylabel("Stage (GPU)")
    ax.set_xlabel("Time Step")
    ax.set_yticks(range(P))
    ax.set_yticklabels([f"GPU {i}" for i in range(P)])

fig.suptitle("Pipeline Schedules — We Will Build These!", fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Car Factory
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_car_factory.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition — The Car Factory

Before we write a single line of code, let us build a mental model.

**Imagine a car factory with four stations:**

| Station | Task | Specialist |
|---------|------|-----------|
| Station 1 | Build the chassis | GPU 0 |
| Station 2 | Add the engine | GPU 1 |
| Station 3 | Paint the body | GPU 2 |
| Station 4 | Install the interior | GPU 3 |

Each station handles one stage of the assembly. A car starts at Station 1, moves to Station 2, then Station 3, and finally Station 4.

**The key insight:** As soon as Station 1 finishes the chassis for Car #1, it can immediately start on Car #2 — it does not need to wait for Car #1 to be fully assembled. Meanwhile, Station 2 begins working on Car #1's engine. This overlap is the whole point of a pipeline.

**But here is the catch.** When only *one* car is being built:
- Station 1 works. Stations 2, 3, 4 sit idle.
- Station 2 works. Stations 1, 3, 4 sit idle.
- Station 3 works. Stations 1, 2, 4 sit idle.
- Station 4 works. Stations 1, 2, 3 sit idle.

Three out of four stations are always idle. That is **75% wasted time**. This idle time is called the **pipeline bubble**.

In [ ]:
#@title 🎧 Listen: Think About This
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_think_about_this.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🤔 Think About This

Before moving on, try to answer these questions in your head:

1. If you have 4 stations and send **100 cars** through the factory, roughly what fraction of time is wasted on idle stations?
2. What if you only send **2 cars**?
3. Is there a way to completely eliminate the bubble?

*Take a minute. Then scroll down.*

In [ ]:
# Let us check our intuition with quick calculations
print("Car Factory Bubble Check:")
print(f"  100 cars, 4 stations: bubble = {3/103*100:.1f}%  (tiny!)")
print(f"    2 cars, 4 stations: bubble = {3/5*100:.1f}%  (bad!)")
print(f"    1 car,  4 stations: bubble = {3/4*100:.1f}% (terrible!)")
print()
print("💡 You can never fully eliminate the bubble (as long as P > 1),")
print("   but you can make it negligibly small with many micro-batches.")

In [ ]:
#@title 🎧 Listen: Mathematics Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_mathematics_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

Now let us formalize this intuition.

### 3.1 The Bubble Fraction

With $P$ pipeline stages and $M$ micro-batches, the pipeline bubble fraction is:

$$\text{Bubble fraction} = \frac{P - 1}{P - 1 + M}$$

This equation says: the "startup cost" is $P - 1$ time steps (waiting for the pipeline to fill up), and the total useful work takes $P - 1 + M$ time steps. The fraction of time wasted is the startup cost divided by the total.

In [ ]:
# Let us plug in some simple numbers to build intuition

print("=" * 55)
print(f"{'P (stages)':<12} {'M (micro-batches)':<20} {'Bubble %':<12} {'Efficiency %'}")
print("=" * 55)

examples = [
    (4, 1),    # Worst case: single micro-batch
    (4, 4),    # Equal to number of stages
    (4, 8),    # Moderate
    (4, 12),   # Good
    (4, 32),   # Excellent
    (8, 1),    # 8 stages, 1 micro-batch
    (8, 32),   # 8 stages, many micro-batches
    (16, 64),  # Large-scale
]

for P, M in examples:
    bubble = (P - 1) / (P - 1 + M)
    efficiency = 1 - bubble
    print(f"{P:<12} {M:<20} {bubble*100:<12.1f} {efficiency*100:.1f}")

print("\n💡 Key takeaway: More micro-batches → smaller bubble!")
print("   The bubble becomes negligible when M >> P.")

In [ ]:
#@title 🎧 Listen: Formula Trace
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_formula_trace.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.2 Understanding the Formula Computationally

Let us trace through exactly what happens with $P = 4$ stages and $M = 1$ micro-batch:

| Time step | GPU 0 | GPU 1 | GPU 2 | GPU 3 |
|-----------|-------|-------|-------|-------|
| t=0       | F(1)  | idle  | idle  | idle  |
| t=1       | idle  | F(1)  | idle  | idle  |
| t=2       | idle  | idle  | F(1)  | idle  |
| t=3       | idle  | idle  | idle  | F(1)  |
| t=4       | idle  | idle  | idle  | B(1)  |
| t=5       | idle  | idle  | B(1)  | idle  |
| t=6       | idle  | B(1)  | idle  | idle  |
| t=7       | B(1)  | idle  | idle  | idle  |

Total cells: $4 \times 8 = 32$. Active cells: $8$ (4 forward + 4 backward). Idle cells: $24$. Bubble = $24/32 = 75\%$.

Now with $M = 4$ micro-batches, many of those idle cells get filled by the next micro-batch's forward or backward pass.

In [ ]:
#@title 🎧 Listen: Memory Analysis
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_memory_analysis.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 3.3 Memory Analysis: GPipe vs 1F1B

There are two major pipeline schedules, and they differ dramatically in **peak memory**:

**GPipe** (all forwards first, then all backwards):
$$\text{Peak memory} = M \times A$$
where $A$ is the activation memory per micro-batch. GPipe must store activations for ALL $M$ micro-batches simultaneously, because no backward pass runs until all forwards are complete.

**1F1B** (interleave one forward, one backward):
$$\text{Peak memory} = P \times A$$
Since backward passes start as soon as possible, activations are released early. In steady state, only $P$ micro-batches worth of activations are stored at once.

In [ ]:
# Let us see the memory difference concretely

print("Memory Comparison: GPipe vs 1F1B")
print("=" * 60)
print(f"{'P':<6} {'M':<6} {'GPipe (×A)':<15} {'1F1B (×A)':<15} {'Savings'}")
print("=" * 60)

scenarios = [
    (4, 4), (4, 8), (4, 16),
    (4, 32), (8, 32), (16, 64),
]

for P, M in scenarios:
    gpipe_mem = M
    onef1b_mem = P
    savings = (1 - onef1b_mem / gpipe_mem) * 100
    print(f"{P:<6} {M:<6} {gpipe_mem:<15} {onef1b_mem:<15} {savings:.0f}% less")

print("\n💡 1F1B has the SAME bubble fraction as GPipe,")
print("   but MUCH lower peak memory!")
print("   This is why 1F1B is used in practice (Megatron-LM, DeepSpeed).")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_build_stages.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Build It — Component by Component

Time to get our hands dirty. We will build a pipeline parallelism simulator step by step.

### 4.1 Defining a Simple Model and Pipeline Stages

Let us create a small model (8 linear layers) and split it into 4 stages of 2 layers each.

In [ ]:
class SimpleLayer(nn.Module):
    """A single linear layer with ReLU — represents one Transformer layer."""
    def __init__(self, dim, layer_id):
        super().__init__()
        self.layer_id = layer_id
        self.linear = nn.Linear(dim, dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.linear(x))


class PipelineStage(nn.Module):
    """A pipeline stage: consecutive layers assigned to one GPU."""
    def __init__(self, layers, stage_id):
        super().__init__()
        self.stage_id = stage_id
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
# Create an 8-layer model, split into 4 stages of 2 layers each
DIM = 64
NUM_LAYERS = 8
NUM_STAGES = 4
LAYERS_PER_STAGE = NUM_LAYERS // NUM_STAGES

all_layers = [SimpleLayer(DIM, i) for i in range(NUM_LAYERS)]

stages = []
for s in range(NUM_STAGES):
    start = s * LAYERS_PER_STAGE
    end = start + LAYERS_PER_STAGE
    stage = PipelineStage(all_layers[start:end], stage_id=s)
    stages.append(stage)

print("Pipeline Stages:")
for s, stage in enumerate(stages):
    layer_ids = [l.layer_id for l in stage.layers]
    params = sum(p.numel() for p in stage.parameters())
    print(f"  Stage {s} (GPU {s}): Layers {layer_ids}, Params: {params:,}")

total = sum(p.numel() for s in stages for p in s.parameters())
print(f"\nTotal parameters: {total:,}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_data_flow.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Running Data Through the Pipeline

Let us see how activations flow from one stage to the next — this is the core of pipeline parallelism.

In [ ]:
# Create a sample input and pass it through the pipeline
batch_size = 4
x = torch.randn(batch_size, DIM)

print("Forward pass through the pipeline:")
print(f"  Input shape: {x.shape}")
print(f"  Input mean: {x.mean().item():.4f}\n")

activation = x
for stage in stages:
    activation = stage(activation)
    print(f"  After Stage {stage.stage_id}: shape={activation.shape}, "
          f"mean={activation.mean().item():.4f}")

print(f"\n✅ Activations flow: Stage 0 → Stage 1 → Stage 2 → Stage 3")
print(f"   In real PP, each arrow is a GPU-to-GPU data transfer.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_naive_schedule.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 The Naive Schedule (No Pipelining)

Before we build clever schedules, let us implement the **worst case**: process one micro-batch all the way through before starting the next. This makes the bubble problem painfully visible.

In [ ]:
def generate_naive_schedule(P, M):
    """
    Naive pipeline schedule: each micro-batch completes fully
    (all forwards + all backwards) before the next starts.

    Returns: (schedule grid, total time steps)
    Values: positive = forward(mb), negative = backward(mb), 0 = idle
    """
    total_time = M * 2 * P
    schedule = np.zeros((P, total_time), dtype=int)

    for m in range(M):
        base_time = m * 2 * P
        # Forward pass: staircase down
        for s in range(P):
            schedule[s][base_time + s] = m + 1
        # Backward pass: staircase up
        for s in range(P):
            schedule[P - 1 - s][base_time + P + s] = -(m + 1)

    return schedule, total_time

In [ ]:
# Generate and display the naive schedule
P, M = 4, 4
naive_schedule, naive_time = generate_naive_schedule(P, M)

total_cells = P * naive_time
active_cells = np.count_nonzero(naive_schedule)
idle_cells = total_cells - active_cells
bubble_pct = idle_cells / total_cells * 100

print(f"Naive Schedule: P={P} stages, M={M} micro-batches")
print(f"  Total time steps: {naive_time}")
print(f"  Total cells: {total_cells}")
print(f"  Active cells: {active_cells}")
print(f"  Idle cells: {idle_cells}")
print(f"  Bubble: {bubble_pct:.1f}%")
print(f"\n⚠️ That is {bubble_pct:.0f}% wasted GPU time — terrible!")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_naive_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


📊 **Visualization Checkpoint 1**: Let us see the naive schedule as a colored grid. First, we need a reusable plotting function.

In [ ]:
def _cell_color_and_label(val, M):
    """Return (color, label) for a schedule cell value."""
    if val > 0:
        return (0.2, 0.4, 0.4 + 0.5*(val/M)), f"F{val}"
    elif val < 0:
        return (0.4 + 0.5*(abs(val)/M), 0.5, 0.2), f"B{abs(val)}"
    return (0.85, 0.85, 0.85), ""


def plot_schedule(schedule, total_time, P, M,
                  title="Pipeline Schedule", ax=None, show_legend=True):
    """Plot a pipeline schedule as a colored grid."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(max(14, total_time*0.5), 3))

    for s in range(P):
        for t in range(total_time):
            color, label = _cell_color_and_label(schedule[s][t], M)
            rect = plt.Rectangle((t, s), 1, 1, facecolor=color,
                                 edgecolor='white', linewidth=1.5)
            ax.add_patch(rect)
            if label:
                ax.text(t+0.5, s+0.5, label, ha='center', va='center',
                        fontsize=7, fontweight='bold', color='white')

    _format_schedule_axes(ax, total_time, P, title, show_legend)
    return ax

In [ ]:
def _format_schedule_axes(ax, total_time, P, title, show_legend):
    """Helper to format axes for schedule plots."""
    ax.set_xlim(0, total_time)
    ax.set_ylim(0, P)
    ax.set_yticks([i + 0.5 for i in range(P)])
    ax.set_yticklabels([f"GPU {i}" for i in range(P)])
    ax.set_xlabel("Time Step", fontsize=11)
    ax.set_ylabel("Stage", fontsize=11)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.invert_yaxis()
    ax.set_aspect('equal')

    if show_legend:
        fwd = mpatches.Patch(color=(0.2, 0.4, 0.7), label='Forward')
        bwd = mpatches.Patch(color=(0.7, 0.5, 0.2), label='Backward')
        idle = mpatches.Patch(color=(0.85, 0.85, 0.85), label='Idle (Bubble)')
        ax.legend(handles=[fwd, bwd, idle], loc='upper right', fontsize=8)

In [ ]:
# 📊 Visualize the naive schedule
fig, ax = plt.subplots(figsize=(16, 3.5))
plot_schedule(naive_schedule, naive_time, P, M,
              title=f"Naive Schedule (P={P}, M={M}) — {bubble_pct:.0f}% Bubble!",
              ax=ax)
plt.tight_layout()
plt.show()

print("Each micro-batch must fully complete before the next one starts.")
print("Most GPUs are idle most of the time. We can do MUCH better.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_gpipe_schedule.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 GPipe Schedule

**GPipe** was the first practical pipeline schedule. The idea: split the mini-batch into $M$ micro-batches, run ALL forward passes first (in a staggered fashion), then run ALL backward passes.

This fills in many of the idle cells from the naive schedule. The forward passes cascade down the stages like a waterfall, and then the backward passes cascade back up.

In [ ]:
def generate_gpipe_schedule(P, M):
    """
    GPipe schedule: all forwards first, then all backwards.
    Micro-batch m enters stage s at time m + s (forward phase).
    """
    fwd_end = M + P - 2
    total_time = 2 * (M + P - 1)
    schedule = np.zeros((P, total_time), dtype=int)

    # Forward passes: micro-batch m enters stage s at time m + s
    for m in range(M):
        for s in range(P):
            schedule[s][m + s] = m + 1

    # Backward passes: run in reverse after all forwards
    bwd_start = fwd_end + 1
    for m in range(M):
        for s in range(P):
            t = bwd_start + (M - 1 - m) + (P - 1 - s)
            schedule[s][t] = -(m + 1)

    return schedule, total_time

In [ ]:
# Generate and analyze the GPipe schedule
P, M = 4, 8
gpipe_schedule, gpipe_time = generate_gpipe_schedule(P, M)

total_cells = P * gpipe_time
active_cells = np.count_nonzero(gpipe_schedule)
idle_cells = total_cells - active_cells
gpipe_bubble_pct = idle_cells / total_cells * 100
theoretical_bubble = (P - 1) / (P - 1 + M) * 100

print(f"GPipe Schedule: P={P} stages, M={M} micro-batches")
print(f"  Total time steps: {gpipe_time}")
print(f"  Active cells: {active_cells} (expected: {2*P*M})")
print(f"  Bubble: {gpipe_bubble_pct:.1f}%")
print(f"  Theoretical bubble fraction: {theoretical_bubble:.1f}%")
print(f"\n💡 Much better than the naive schedule!")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_gpipe_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


📊 **Visualization Checkpoint 2**: The GPipe schedule grid.

In [ ]:
# 📊 Visualize the GPipe schedule
fig, ax = plt.subplots(figsize=(18, 4))
plot_schedule(gpipe_schedule, gpipe_time, P, M,
              title=f"GPipe Schedule (P={P}, M={M}) — Bubble: {gpipe_bubble_pct:.1f}%",
              ax=ax)
plt.tight_layout()
plt.show()

print("📊 Notice the pattern:")
print("  - All forward passes (blue) run first in a staircase")
print("  - Then all backward passes (orange) in a reverse staircase")
print("  - Gray cells in the corners are the bubble")
print(f"\n⚠️ Peak memory: GPipe stores activations for ALL {M} micro-batches = {M}×A")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_1f1b_schedule.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 The 1F1B Schedule

**1F1B (One-Forward-One-Backward)** is the schedule used by Megatron-LM and most production systems. The idea is to **interleave** forward and backward passes so that activations can be released as soon as their corresponding backward pass completes.

The schedule has three phases:

1. **Warmup phase**: Fill the pipeline with forward passes (each stage does progressively fewer warmup forwards)
2. **Steady state**: Alternate one forward and one backward pass
3. **Cooldown phase**: Drain remaining backward passes

In [ ]:
def generate_1f1b_schedule(P, M):
    """1F1B: warmup forwards, then alternate 1F+1B, then drain backwards."""
    total_time = 2 * (M + P - 1)
    schedule = np.zeros((P, total_time), dtype=int)

    for s in range(P):
        num_warmup = P - 1 - s
        t, fwd_idx, bwd_idx = s, 0, 0

        for _ in range(num_warmup):  # Phase 1: warmup
            if fwd_idx < M:
                schedule[s][t] = fwd_idx + 1; fwd_idx += 1; t += 1

        while fwd_idx < M:  # Phase 2: steady state (1F, 1B)
            schedule[s][t] = fwd_idx + 1; fwd_idx += 1; t += 1
            if bwd_idx < M:
                schedule[s][t] = -(bwd_idx + 1); bwd_idx += 1; t += 1

        while bwd_idx < M:  # Phase 3: cooldown
            schedule[s][t] = -(bwd_idx + 1); bwd_idx += 1; t += 1

    return schedule, total_time

In [ ]:
# Generate and analyze the 1F1B schedule
P, M = 4, 8
f1b_schedule, f1b_time = generate_1f1b_schedule(P, M)

total_cells = P * f1b_time
active_cells = np.count_nonzero(f1b_schedule)
idle_cells = total_cells - active_cells
f1b_bubble_pct = idle_cells / total_cells * 100

print(f"1F1B Schedule: P={P} stages, M={M} micro-batches")
print(f"  Total time steps: {f1b_time}")
print(f"  Active cells: {active_cells} (expected: {2*P*M})")
print(f"  Bubble: {f1b_bubble_pct:.1f}%")
print(f"\n💡 Same bubble fraction as GPipe ({gpipe_bubble_pct:.1f}%)")
print(f"   But peak memory: 1F1B stores only P={P} activations vs GPipe's M={M}")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_side_by_side.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


📊 **Visualization Checkpoint 3**: Side-by-side comparison of GPipe and 1F1B.

In [ ]:
# 📊 Side-by-side comparison: GPipe vs 1F1B
P, M = 4, 8
gpipe_sched, gpipe_t = generate_gpipe_schedule(P, M)
f1b_sched, f1b_t = generate_1f1b_schedule(P, M)

fig, axes = plt.subplots(2, 1, figsize=(18, 7))

plot_schedule(gpipe_sched, gpipe_t, P, M,
              title=f"GPipe (P={P}, M={M}) — Peak Memory: {M}×A",
              ax=axes[0], show_legend=True)

plot_schedule(f1b_sched, f1b_t, P, M,
              title=f"1F1B (P={P}, M={M}) — Peak Memory: {P}×A",
              ax=axes[1], show_legend=True)

plt.tight_layout()
plt.show()

print("Key observations:")
print(f"  ✅ Both have the same bubble fraction: {(P-1)/(P-1+M)*100:.1f}%")
print(f"  ✅ 1F1B peak memory: {P}×A  vs  GPipe: {M}×A")
print(f"  ✅ Memory savings: {(1 - P/M)*100:.0f}% less with 1F1B!")

In [ ]:
#@title 🎧 Listen: Memory Deep Dive
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_memory_deep_dive.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.6 Understanding Peak Memory — A Deeper Look

Why exactly does 1F1B use less memory? Let us trace the activation storage over time for each schedule.

In [ ]:
def compute_memory_profile(schedule, total_time, P, M):
    """
    Track stored activations over time.
    Forward pass creates +1 activation; backward pass releases -1.
    """
    memory_timeline = []
    stored = [0] * P

    for t in range(total_time):
        for s in range(P):
            val = schedule[s][t]
            if val > 0:
                stored[s] += 1
            elif val < 0:
                stored[s] = max(0, stored[s] - 1)
        memory_timeline.append(sum(stored))

    return memory_timeline

In [ ]:
# Compute and plot memory profiles
P, M = 4, 8
gpipe_sched, gpipe_t = generate_gpipe_schedule(P, M)
f1b_sched, f1b_t = generate_1f1b_schedule(P, M)

gpipe_memory = compute_memory_profile(gpipe_sched, gpipe_t, P, M)
f1b_memory = compute_memory_profile(f1b_sched, f1b_t, P, M)

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(range(gpipe_t), gpipe_memory, 'b-o', markersize=4,
        label=f'GPipe (peak={max(gpipe_memory)}×A)', linewidth=2)
ax.plot(range(f1b_t), f1b_memory, 'r-s', markersize=4,
        label=f'1F1B (peak={max(f1b_memory)}×A)', linewidth=2)

ax.axhline(y=M, color='blue', linestyle='--', alpha=0.5,
           label=f'M={M} (GPipe theoretical)')
ax.axhline(y=P, color='red', linestyle='--', alpha=0.5,
           label=f'P={P} (1F1B theoretical)')

ax.set_xlabel("Time Step", fontsize=12)
ax.set_ylabel("Stored Activations (×A)", fontsize=12)
ax.set_title("Peak Memory: GPipe vs 1F1B", fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print(f"📊 GPipe peak memory:  {max(gpipe_memory)} activations")
print(f"📊 1F1B peak memory:   {max(f1b_memory)} activations")
savings_pct = (1 - max(f1b_memory)/max(gpipe_memory))*100
print(f"📊 Memory savings:     {savings_pct:.0f}%")
print(f"\n💡 In GPipe, memory ramps up during the forward phase because")
print(f"   ALL activations are stored before any backward pass runs.")
print(f"   In 1F1B, backwards start early, releasing memory sooner.")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. 🔧 Your Turn

Now it is time for you to implement some key components yourself.

### 🔧 TODO 1: Implement the Bubble Fraction Calculator

Given the number of pipeline stages $P$ and micro-batches $M$, compute the bubble fraction and the pipeline efficiency.

In [ ]:
def compute_bubble_fraction(P, M):
    """
    Compute the pipeline bubble fraction and efficiency.

    Args:
        P: Number of pipeline stages
        M: Number of micro-batches

    Returns:
        tuple: (bubble_fraction, efficiency)
    """
    # ============ TODO ============
    # Implement the bubble fraction formula:
    #   bubble = (P - 1) / (P - 1 + M)
    #   efficiency = 1 - bubble
    #
    # Edge cases:
    #   - M = 0? Return bubble = 1.0
    #   - P = 1? Return bubble = 0.0
    # ==============================

    bubble_fraction = ???  # YOUR CODE HERE
    efficiency = ???       # YOUR CODE HERE

    return bubble_fraction, efficiency

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_todo1_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# ✅ Verification: Run this cell to check your implementation

test_cases = [
    (4, 1, 0.75),      (4, 4, 3/7),
    (4, 12, 0.20),     (4, 32, 3/35),
    (8, 1, 7/8),       (1, 10, 0.0),
    (8, 0, 1.0),
]

all_passed = True
for P, M, expected in test_cases:
    bubble, eff = compute_bubble_fraction(P, M)
    match = abs(bubble - expected) < 1e-6
    status = "✅" if match else "❌"
    if not match:
        all_passed = False
    print(f"  {status} P={P}, M={M}: bubble={bubble:.4f} "
          f"(expected {expected:.4f}), efficiency={eff:.4f}")

if all_passed:
    print("\n✅ All tests passed! Great work!")
else:
    print("\n❌ Some tests failed. Check your formula.")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_todo2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🔧 TODO 2: Implement the GPipe Forward Schedule

Given $P$ stages and $M$ micro-batches, generate the **forward phase** of the GPipe schedule. In GPipe, micro-batch $m$ (0-indexed) enters stage $s$ at time step $m + s$.

In [ ]:
def generate_gpipe_forward_schedule(P, M):
    """
    Generate ONLY the forward phase of the GPipe schedule.

    Micro-batch m (0-indexed) enters stage s at time step m + s.

    Returns:
        schedule: array of shape (P, M+P-1), values are micro-batch
                  numbers (1-indexed) or 0 for idle
        total_fwd_time: total time steps in forward phase
    """
    total_fwd_time = M + P - 1
    schedule = np.zeros((P, total_fwd_time), dtype=int)

    # ============ TODO ============
    # For each micro-batch m (0 to M-1):
    #   For each stage s (0 to P-1):
    #     Set schedule[s][m + s] = m + 1
    # ==============================

    # YOUR CODE HERE

    return schedule, total_fwd_time

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_todo2_verify.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# ✅ Verification: Run this cell to check your implementation

P_test, M_test = 4, 6
fwd_sched, fwd_time = generate_gpipe_forward_schedule(P_test, M_test)

assert fwd_sched.shape == (P_test, fwd_time), \
    f"❌ Shape mismatch: {fwd_sched.shape} vs ({P_test}, {fwd_time})"

total_fwd = np.count_nonzero(fwd_sched)
assert total_fwd == P_test * M_test, \
    f"❌ Total forwards: {total_fwd}, expected {P_test * M_test}"

for s in range(P_test):
    for m in range(M_test):
        assert fwd_sched[s][m + s] == m + 1, \
            f"❌ Stage {s}, time {m+s}: got {fwd_sched[s][m+s]}, expected {m+1}"

print("Your GPipe forward schedule:")
for s in range(P_test):
    row = ""
    for t in range(fwd_time):
        val = fwd_sched[s][t]
        row += f" F{val}" if val > 0 else "  . "
    print(f"  GPU {s}: {row}")

print(f"\n✅ All checks passed! Your GPipe forward schedule is correct.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/20_training_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together: Training with Pipeline Parallelism

### 6.1 Simulated Pipeline Training

Let us actually train a small model using pipeline-style forward and backward passes. We will simulate the pipeline by running stages sequentially (on a single GPU), but tracking the behavior as if they were on separate GPUs.

In [ ]:
# Create a fresh model split into pipeline stages
DIM = 32
NUM_LAYERS = 8
NUM_STAGES = 4
LAYERS_PER_STAGE = NUM_LAYERS // NUM_STAGES
NUM_MICROBATCHES = 4
BATCH_SIZE = 32
MICROBATCH_SIZE = BATCH_SIZE // NUM_MICROBATCHES

all_layers = [SimpleLayer(DIM, i) for i in range(NUM_LAYERS)]
stages = []
for s in range(NUM_STAGES):
    start = s * LAYERS_PER_STAGE
    end = start + LAYERS_PER_STAGE
    stage = PipelineStage(all_layers[start:end], stage_id=s)
    stages.append(stage)

optimizers = [torch.optim.SGD(stage.parameters(), lr=0.01) for stage in stages]

print(f"Model: {NUM_LAYERS} layers, split into {NUM_STAGES} stages")
print(f"Batch size: {BATCH_SIZE}, Micro-batches: {NUM_MICROBATCHES}")
print(f"Micro-batch size: {MICROBATCH_SIZE}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/21_training_phases.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now let us define the GPipe-style training step. This function runs all forward passes first (storing activations), then all backward passes.

In [ ]:
def pipeline_forward_phase(stages, input_mbs, M, P):
    """Phase 1 of GPipe: run ALL forward passes, store activations."""
    activations = [[None] * (P + 1) for _ in range(M)]

    for m in range(M):
        activations[m][0] = input_mbs[m].detach().requires_grad_(True)
        for s in range(P):
            inp = activations[m][s]
            if s > 0:
                inp = inp.detach().requires_grad_(True)
                activations[m][s] = inp
            activations[m][s + 1] = stages[s](inp)

    return activations

In [ ]:
def pipeline_backward_phase(stages, activations, target_mbs, M, P):
    """Phase 2 of GPipe: run ALL backward passes in reverse order."""
    losses = []

    for m in reversed(range(M)):
        output = activations[m][P]
        loss = ((output - target_mbs[m]) ** 2).mean()
        losses.append(loss.item())
        loss.backward()

    return losses

Now we combine the forward and backward phases into one training step, then run training.

In [ ]:
def pipeline_train_step(stages, optimizers, inputs, targets, num_mb):
    """Full GPipe training step: forward phase → backward phase → update."""
    P = len(stages)
    M = num_mb
    mb_size = inputs.shape[0] // M

    input_mbs = inputs.split(mb_size)
    target_mbs = targets.split(mb_size)

    activations = pipeline_forward_phase(stages, input_mbs, M, P)
    losses = pipeline_backward_phase(stages, activations, target_mbs, M, P)

    for opt in optimizers:
        opt.step()
        opt.zero_grad()

    return np.mean(losses)

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/22_training_run.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Train for several steps and track the loss
num_steps = 50
loss_history = []

for step in range(num_steps):
    inputs = torch.randn(BATCH_SIZE, DIM)
    targets = inputs * 0.5 + 0.1  # Simple regression target

    loss = pipeline_train_step(stages, optimizers, inputs, targets,
                               NUM_MICROBATCHES)
    loss_history.append(loss)

    if step % 10 == 0:
        print(f"  Step {step:3d}: Loss = {loss:.4f}")

print(f"\n✅ Training complete! Loss: {loss_history[0]:.4f} → {loss_history[-1]:.4f}")

In [ ]:
# 📊 Plot the training loss
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(loss_history, 'b-', linewidth=2, alpha=0.8)
ax.set_xlabel("Training Step", fontsize=12)
ax.set_ylabel("Loss (MSE)", fontsize=12)
ax.set_title("Pipeline-Parallel Training: Loss Over Time",
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

ax.annotate(f"Start: {loss_history[0]:.3f}", xy=(0, loss_history[0]),
            fontsize=10, color='red', fontweight='bold')
ax.annotate(f"End: {loss_history[-1]:.3f}",
            xy=(len(loss_history)-1, loss_history[-1]),
            fontsize=10, color='green', fontweight='bold')

plt.tight_layout()
plt.show()

print("💡 The model successfully trains with pipeline parallelism!")
print("   Each step splits the batch into micro-batches and")
print("   runs them through the 4-stage pipeline.")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/23_comprehensive_comparison.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 6.2 Comprehensive Schedule Comparison

Let us now do a thorough comparison across different configurations.

In [ ]:
# 📊 Big comparison across different P and M values

configs = [
    (4, 4), (4, 8), (4, 16),
    (8, 8), (8, 16), (8, 32),
]

fig, axes = plt.subplots(len(configs), 2,
                         figsize=(20, 3.5 * len(configs)))

for idx, (P, M) in enumerate(configs):
    gpipe_s, gpipe_t = generate_gpipe_schedule(P, M)
    f1b_s, f1b_t = generate_1f1b_schedule(P, M)
    bubble = (P - 1) / (P - 1 + M) * 100

    plot_schedule(gpipe_s, gpipe_t, P, M,
                  title=f"GPipe (P={P}, M={M}) — Bubble: {bubble:.1f}%",
                  ax=axes[idx][0], show_legend=(idx == 0))
    plot_schedule(f1b_s, f1b_t, P, M,
                  title=f"1F1B (P={P}, M={M}) — Bubble: {bubble:.1f}%",
                  ax=axes[idx][1], show_legend=(idx == 0))

plt.suptitle("GPipe vs 1F1B Across Configurations",
             fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Summary table
print("\n" + "=" * 70)
print(f"{'Config':<12} {'Bubble %':<11} {'GPipe Mem':<13} {'1F1B Mem':<13} {'Savings'}")
print("=" * 70)

for P, M in configs:
    bubble = (P - 1) / (P - 1 + M) * 100
    savings = (1 - P / M) * 100
    print(f"P={P}, M={M:<4} {bubble:<11.1f} {M}×A{'':<9} {P}×A{'':<9} {savings:.0f}%")

print("=" * 70)
print("\n💡 Both schedules have the SAME bubble fraction.")
print("   1F1B always wins on memory (when M > P).")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/24_simulator_build.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. 🎯 Final Output: Interactive Pipeline Simulator

Now let us put everything together into a complete, interactive simulator. The student can change the number of stages ($P$) and micro-batches ($M$) to see how the schedule, bubble fraction, and memory usage change.

In [ ]:
def draw_polished_grid(ax, schedule, total_time, P, M, title):
    """Draw a polished schedule grid with matplotlib colormaps."""
    for s in range(P):
        for t in range(total_time):
            val = schedule[s][t]
            if val > 0:
                color = plt.cm.Blues(0.35 + 0.55*(val/M)); lbl = f"F{val}"
            elif val < 0:
                color = plt.cm.Oranges(0.35 + 0.55*(abs(val)/M)); lbl = f"B{abs(val)}"
            else:
                color = '#E8E8E8'; lbl = ""
            ax.add_patch(plt.Rectangle((t,s), 1, 1, facecolor=color,
                                       edgecolor='white', linewidth=1.2))
            if lbl and total_time <= 30:
                ax.text(t+0.5, s+0.5, lbl, ha='center', va='center',
                        fontsize=6 if total_time > 20 else 7,
                        fontweight='bold', color='white')

    ax.set_xlim(0, total_time); ax.set_ylim(0, P)
    ax.set_yticks([i+0.5 for i in range(P)])
    ax.set_yticklabels([f"GPU {i}" for i in range(P)])
    ax.set_xlabel("Time Step", fontsize=10)
    ax.set_title(title, fontsize=12, fontweight='bold', pad=10)
    ax.invert_yaxis(); ax.set_aspect('equal')

In [ ]:
def pipeline_simulator(P, M):
    """Full pipeline simulator: schedules + memory + metrics."""
    gpipe_s, gpipe_t = generate_gpipe_schedule(P, M)
    f1b_s, f1b_t = generate_1f1b_schedule(P, M)
    bubble = (P-1)/(P-1+M); eff = 1-bubble
    mem_sav = (1-P/M)*100 if M > 0 else 0
    gm = compute_memory_profile(gpipe_s, gpipe_t, P, M)
    fm = compute_memory_profile(f1b_s, f1b_t, P, M)

    fig = plt.figure(figsize=(20, 14))
    gs = fig.add_gridspec(3, 2, hspace=0.45, wspace=0.3,
                          height_ratios=[1, 1, 0.8])
    draw_polished_grid(fig.add_subplot(gs[0,:]), gpipe_s, gpipe_t,
                       P, M, f"GPipe Schedule (P={P}, M={M})")
    draw_polished_grid(fig.add_subplot(gs[1,:]), f1b_s, f1b_t,
                       P, M, f"1F1B Schedule (P={P}, M={M})")
    _draw_memory_panel(fig.add_subplot(gs[2,0]),
                       gm, fm, gpipe_t, f1b_t, P, M)
    _draw_summary_panel(fig.add_subplot(gs[2,1]), bubble, eff, M, P)
    _add_legend(fig)
    fig.suptitle(f"Pipeline Simulator — P={P}, M={M}",
                 fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    _print_summary(P, M, bubble, eff, mem_sav)

In [ ]:
def _draw_memory_panel(ax, gpipe_mem, f1b_mem, gpipe_t, f1b_t, P, M):
    """Draw the memory profile comparison panel."""
    ax.plot(range(gpipe_t), gpipe_mem, 'b-o', ms=3,
            label=f'GPipe (peak={max(gpipe_mem)})', lw=2)
    ax.plot(range(f1b_t), f1b_mem, 'r-s', ms=3,
            label=f'1F1B (peak={max(f1b_mem)})', lw=2)
    ax.axhline(y=M, color='blue', ls='--', alpha=0.3)
    ax.axhline(y=P, color='red', ls='--', alpha=0.3)
    ax.set_xlabel("Time Step")
    ax.set_ylabel("Stored Activations")
    ax.set_title("Memory Profile", fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)


def _draw_summary_panel(ax, bubble_frac, efficiency, M, P):
    """Draw the summary metrics bar chart."""
    labels = ['Bubble\n(%)', 'Efficiency\n(%)',
              'GPipe Mem\n(×A)', '1F1B Mem\n(×A)']
    values = [bubble_frac * 100, efficiency * 100, M, P]
    colors = ['#FF6B6B', '#4CAF50', '#2196F3', '#FF9800']
    bars = ax.bar(labels, values, color=colors,
                  edgecolor='white', lw=2)
    for bar, val, lbl in zip(bars, values, labels):
        txt = f"{int(val)}×A" if "Mem" in lbl else f"{val:.1f}%"
        ax.text(bar.get_x() + bar.get_width()/2,
                bar.get_height() + 0.5, txt,
                ha='center', va='bottom', fontweight='bold')
    ax.set_title("Summary Metrics", fontweight='bold')
    ax.grid(True, alpha=0.2, axis='y')

These small helpers handle the figure legend and text summary output.

In [ ]:
def _add_legend(fig):
    """Add a shared legend to the figure."""
    fwd = mpatches.Patch(facecolor=plt.cm.Blues(0.6), label='Forward')
    bwd = mpatches.Patch(facecolor=plt.cm.Oranges(0.6), label='Backward')
    idle = mpatches.Patch(facecolor='#E8E8E8', edgecolor='gray',
                          label='Idle')
    fig.legend(handles=[fwd, bwd, idle], loc='upper right',
               fontsize=10, framealpha=0.9)


def _print_summary(P, M, bubble_frac, efficiency, mem_savings):
    """Print text summary of pipeline metrics."""
    print(f"\n{'='*60}")
    print(f"  PIPELINE PARALLELISM SUMMARY")
    print(f"{'='*60}")
    print(f"  Stages (P):           {P}")
    print(f"  Micro-batches (M):    {M}")
    print(f"  Bubble fraction:      {bubble_frac*100:.1f}%")
    print(f"  Pipeline efficiency:  {efficiency*100:.1f}%")
    print(f"  GPipe peak memory:    {M}×A")
    print(f"  1F1B peak memory:     {P}×A")
    print(f"  Memory savings (1F1B):{mem_savings:.0f}%")
    print(f"{'='*60}")

Everything is defined. Let us run the simulator!

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/25_simulator_run.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🎯 Run the full simulator with default settings!
pipeline_simulator(P=4, M=8)

In [ ]:
#@title 🎧 Listen: Experiments
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/26_experiments.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Now try different configurations! Change P and M below to explore how the pipeline schedule changes.

In [ ]:
# 🎯 Experiment 1: Few micro-batches (large bubble)
print("=" * 60)
print("  EXPERIMENT 1: Few micro-batches → large bubble")
print("=" * 60)
pipeline_simulator(P=4, M=4)

💡 Notice how the bubble shrinks as we increase $M$. Let us try doubling it.

In [ ]:
# 🎯 Experiment 2: Many micro-batches (small bubble)
print("=" * 60)
print("  EXPERIMENT 2: Many micro-batches → small bubble")
print("=" * 60)
pipeline_simulator(P=4, M=16)

Now let us scale up to 8 stages — closer to what production systems use.

In [ ]:
# 🎯 Experiment 3: Many stages (realistic scale)
print("=" * 60)
print("  EXPERIMENT 3: 8 stages with 32 micro-batches")
print("=" * 60)
pipeline_simulator(P=8, M=32)

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/27_bubble_explorer.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🎯 Bubble Fraction Explorer

Let us create one more visualization: how the bubble fraction changes as we increase the number of micro-batches for different numbers of stages.

In [ ]:
# 📊 Bubble fraction as a function of M for various P
fig, ax = plt.subplots(figsize=(10, 6))
M_values = np.arange(1, 65)

for P in [2, 4, 8, 16]:
    bubble_fracs = [(P-1) / (P-1+M) * 100 for M in M_values]
    ax.plot(M_values, bubble_fracs, linewidth=2.5, label=f'P={P}')

ax.axhline(y=10, color='gray', linestyle='--', alpha=0.5, label='10% line')
ax.set_xlabel("Number of Micro-batches (M)", fontsize=12)
ax.set_ylabel("Bubble Fraction (%)", fontsize=12)
ax.set_title("Bubble Fraction vs Micro-batches",
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.show()

In [ ]:
# 📊 Memory savings of 1F1B over GPipe
fig, ax = plt.subplots(figsize=(10, 6))

for P in [2, 4, 8, 16]:
    M_valid = [m for m in M_values if m >= P]
    savings = [(1 - P / m) * 100 for m in M_valid]
    ax.plot(M_valid, savings, linewidth=2.5, label=f'P={P}')

ax.set_xlabel("Number of Micro-batches (M)", fontsize=12)
ax.set_ylabel("Memory Savings (%)", fontsize=12)
ax.set_title("1F1B Memory Savings over GPipe",
             fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.show()

In [ ]:
# 💡 Minimum M needed for <10% bubble at each P
print("💡 Minimum micro-batches for <10% bubble:")
for P in [2, 4, 8, 16]:
    M_min = int(np.ceil(9 * (P - 1)))
    actual_bubble = (P - 1) / (P - 1 + M_min) * 100
    print(f"   P={P:<3} → M ≥ {M_min:<3} (bubble = {actual_bubble:.1f}%)")

print("\n🎉 Congratulations! You have built a pipeline parallelism")
print("   simulator from scratch!")
print()
print("   You now understand:")
print("   ✅ Why pipeline parallelism exists (split model by depth)")
print("   ✅ The bubble problem and its formula")
print("   ✅ GPipe schedule (all F then all B)")
print("   ✅ 1F1B schedule (interleaved F and B)")
print("   ✅ Why 1F1B has the same bubble but lower memory")
print("   ✅ How to choose M to minimize the bubble")

In [ ]:
#@title 🎧 Listen: Reflection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/28_reflection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Reflection and Next Steps

### 🤔 Reflection Questions

1. **Why does 1F1B have lower peak memory than GPipe despite having the same bubble fraction?**
   *Hint: Think about when activations are created and when they are released.*

2. **What is the minimum number of micro-batches needed to keep the bubble below 10% with 8 pipeline stages?**
   *Hint: Solve $(P - 1) / (P - 1 + M) < 0.10$ for $M$.*

3. **In practice, PP is used across nodes while TP is within a node. Why?**
   *Hint: Think about communication frequency. PP communicates at stage boundaries (once per micro-batch per layer boundary). TP communicates within every single layer. Which one can tolerate higher latency?*

4. **Why do we split a mini-batch into micro-batches instead of just using a larger pipeline depth?**
   *Hint: More stages means each stage does less work, which increases the relative cost of communication.*

### 🏆 Optional Challenges

For those who want to go deeper:

1. **Interleaved 1F1B**: In the interleaved schedule (used by Megatron-LM v3), each GPU handles *non-contiguous* stages. For example, with 8 stages and 4 GPUs, GPU 0 handles stages 0 and 4, GPU 1 handles stages 1 and 5, etc. This reduces the bubble by a factor of $v$ (the number of virtual stages per GPU). Try implementing this!

2. **Gradient Accumulation**: Modify the training loop to accumulate gradients across micro-batches before updating weights. This is how real PP systems work — the optimizer step only happens once per mini-batch, not once per micro-batch.

3. **Zero-Bubble Pipeline**: Recent research (Qi et al., 2024) proposes schedules that achieve near-zero bubble by overlapping the backward pass for weights (B) and the backward pass for activations (W). Read the paper and think about how this works.

4. **PipeDream-2BW (Double-Buffered Weights)**: The original PipeDream maintained multiple versions of model weights to avoid stalls. PipeDream-2BW improved this by keeping at most 2 versions. How does this trade off memory for pipeline efficiency?

In [ ]:
# 🤔 Quick exercise: answer Reflection Question 2 computationally
P_q2 = 8
target_bubble = 0.10
# Solve: (P-1)/(P-1+M) < 0.10 => M > 9*(P-1)
M_needed = int(np.ceil(9 * (P_q2 - 1)))
actual = (P_q2 - 1) / (P_q2 - 1 + M_needed) * 100
print(f"Q2: With P={P_q2}, need M ≥ {M_needed} for <10% bubble")
print(f"    Actual bubble at M={M_needed}: {actual:.1f}%")

In [ ]:
#@title 🎧 Wrap-Up
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/29_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### What is Next?

In this notebook, we built Pipeline Parallelism from scratch. We saw how splitting a model by depth creates an assembly line of GPUs, and how the bubble fraction is the central tradeoff.

In the **next notebook**, we will explore **Sequence Parallelism** — how to split along the sequence length dimension. This is critical for training models on long contexts (128K+ tokens).

**The journey so far:**
- Notebook 1: Introduction to 5D Parallelism
- Notebook 2: Why Do We Need Parallelism?
- Notebook 3: Data Parallelism (Hire More Chefs)
- **Notebook 4: Pipeline Parallelism (The Assembly Line)** -- You are here!
- Notebook 5: Sequence Parallelism (Split the Sentence)
- Notebook 6: Expert Parallelism (Specialist Chefs)

In [ ]:
# 🎯 Final summary
print("=" * 60)
print("  🚀 PIPELINE PARALLELISM — KEY TAKEAWAYS")
print("=" * 60)
print()
print("  1. PP splits the model by DEPTH")
print("     GPU 0: layers 1-8, GPU 1: layers 9-16, ...")
print()
print("  2. The bubble fraction = (P-1) / (P-1+M)")
print("     More micro-batches → smaller bubble")
print()
print("  3. GPipe: all forwards, then all backwards")
print("     Simple but high peak memory (M × A)")
print()
print("  4. 1F1B: interleave forwards and backwards")
print("     Same bubble, but peak memory = P × A")
print()
print("  5. PP is used ACROSS nodes (tolerates latency)")
print("     TP is used WITHIN a node (needs fast NVLink)")
print()
print("=" * 60)
print("  🎉 Great work! See you in the next notebook.")
print("=" * 60)